In [1]:
from app import db
from db import DeadManPackage, Connectivity
import pandas as pd
from  analytics.connection import connection, _connection_for_device, connection_timeseries, connection_data_per_day
from analytics.sensors import get_sensor_data
from datetime import timedelta, date, time
from utils.date import start_of_day, end_of_day
import numpy as np
from utils.interval import find_intervals
from analytics.mouse import get_mouse_data_raw, get_mouse_data_aggregated
from clustering.clustering import get_input_data, plot_distributions, input_data_clustering, cluster_palette
import seaborn as sns
from sklearn.model_selection import train_test_split
from matplotlib.figure import Figure

from bokeh.models import ColumnDataSource
from bokeh.plotting import show, output_notebook, figure
from bokeh.palettes import Category20c
from bokeh import palettes
from bokeh import layouts
from bokeh.transform import cumsum
output_notebook()

Loading BokehJS ...

In [2]:
device = "PTL_RD_AT_003"
d1 = date(2020, 2, 1)
d2 = date.today() - timedelta(days=1)

In [3]:
from analytics.instruction import get_power

In [4]:
power_data = get_power(device, d1, resample_rule='1Min')
power_data

,power
timestamp,
2020-02-07 10:06:00,0.000000
2020-02-07 10:07:00,0.883333
2020-02-07 10:08:00,1.000000
2020-02-07 10:09:00,0.000000
2020-02-07 10:10:00,0.000000
...,...
2020-05-18 23:55:00,0.000000
2020-05-18 23:56:00,0.000000
2020-05-18 23:57:00,0.000000


In [8]:
from bokeh.palettes import Magma256

In [9]:
len(Magma256)

256

In [9]:
from ajax_plots import PlotPowerTimeline

In [10]:
pp = PlotPowerTimeline(plot_parameters={device: device})

In [11]:
data = pp._fetch()

Fetch data for None


In [12]:
pp.device

In [13]:
device

'PTL_RD_AT_003'

In [14]:
from analytics.connection import connection_timeseries

In [15]:
c_data = connection_timeseries(device, d1, d2)

[2020-05-15 16:35:14] DEBUG(root) in db: db_cached: name=analytics.connection._connection_for_device, parameter_digest=8a5310de436bd690b2a527d29d2db6c62bd0c6cf717930fccd050a1815087ed9
[2020-05-15 16:35:14] DEBUG(root) in db: db_cached: found cached value.
[2020-05-15 16:35:14] DEBUG(root) in db: db_cached: no update needed: return cached data.


In [17]:
c_data.resample("1Min").mean()

,connected
timestamp,
2020-03-12 12:26:00,1.0
2020-03-12 12:27:00,1.0
2020-03-12 12:28:00,1.0
2020-03-12 12:29:00,1.0
2020-03-12 12:30:00,1.0
...,...
2020-05-14 23:55:00,1.0
2020-05-14 23:56:00,1.0
2020-05-14 23:57:00,1.0


In [5]:
from analytics.connection import connection_data_per_day

In [8]:
connection_data = connection_data_per_day(device, d1, d2)

[2020-05-15 16:51:52] DEBUG(root) in db: db_cached: name=analytics.connection.connection_data_per_day, parameter_digest=573e3314b55be968fd15e1a5a83ba794bd0d4d4d5b948f525ddf88406ce66edc
[2020-05-15 16:51:52] DEBUG(root) in db: db_cached: found cached value.
[2020-05-15 16:51:52] DEBUG(root) in db: db_cached: no update needed: return cached data.


In [7]:
from analytics.instruction import get_power

In [9]:
power_data = get_power(device, d1, d2, resample_rule="1Min")

In [10]:
power_data['date'] = power_data.index.date

In [18]:
power_data.merge(connection_data, on=date, how='inner')

KeyError: <class 'datetime.date'>

In [14]:
connection_data = connection_data.reset_index()

In [23]:
connection_data

,date,connected,datalosses,excluded
0,2020-02-01,1.0,0,0
1,2020-02-02,1.0,0,0
2,2020-02-03,1.0,0,0
3,2020-02-04,1.0,0,0
4,2020-02-05,1.0,0,0
...,...,...,...,...
99,2020-05-10,1.0,0,0
100,2020-05-11,1.0,0,0
101,2020-05-12,1.0,0,0
102,2020-05-13,1.0,0,0


In [21]:
power_data

,power,date
timestamp,,
2020-02-07 10:06:00,0.000000,2020-02-07
2020-02-07 10:07:00,0.883333,2020-02-07
2020-02-07 10:08:00,1.000000,2020-02-07
2020-02-07 10:09:00,0.000000,2020-02-07
2020-02-07 10:10:00,0.000000,2020-02-07
...,...,...
2020-05-14 23:55:00,0.000000,2020-05-14
2020-05-14 23:56:00,0.000000,2020-05-14
2020-05-14 23:57:00,0.000000,2020-05-14


In [28]:
included_dates = connection_data[connection_data.excluded==0].date

In [29]:
power_data[power_data.date.isin(included_dates)]

,power,date
timestamp,,
2020-02-07 10:06:00,0.000000,2020-02-07
2020-02-07 10:07:00,0.883333,2020-02-07
2020-02-07 10:08:00,1.000000,2020-02-07
2020-02-07 10:09:00,0.000000,2020-02-07
2020-02-07 10:10:00,0.000000,2020-02-07
...,...,...
2020-05-14 23:55:00,0.000000,2020-05-14
2020-05-14 23:56:00,0.000000,2020-05-14
2020-05-14 23:57:00,0.000000,2020-05-14


In [47]:
def input_data_clustering(device: str, start_date: date, end_date = None,
                          n_clusters=5,
                          return_only_cluster=True,
                          return_pca=False) -> pd.DataFrame:
    def add_column_postfix(df: pd.DataFrame, postfix: str) -> pd.DataFrame:
        columns = df.columns
        mapping = {c: f"{c}_{postfix}" for c in columns}
        return df.rename(columns=mapping)

    # get normalized input data
    data = get_input_data(device, start_date, end_date=end_date, normalized=True)
    if data.empty:
        return data

    # compute statistics over rolling window
    rolling = data.rolling('15Min', min_periods=1, win_type=None)
    data_rolling_ = list()
    data_rolling_.append(add_column_postfix(rolling.count(), "count"))
    data_rolling_.append(add_column_postfix(rolling.sum(), "sum"))
    data_rolling_.append(add_column_postfix(rolling.mean(), "mean"))
    data_rolling_.append(add_column_postfix(rolling.median(), "median"))
    data_rolling_.append(add_column_postfix(rolling.var(), "var"))
    data_rolling_.append(add_column_postfix(rolling.kurt(), "kurt"))
    data_rolling_.append(add_column_postfix(rolling.skew(), "skew"))
    data_rolling = pd.concat(data_rolling_, axis=1)
    data_rolling = data_rolling.loc[~data_rolling.index.duplicated(keep='first')]
    data_rolling = data_rolling.resample("1Min").nearest(limit=1)
    return data_rolling

In [48]:
data_rolling = input_data_clustering(device, d1)

In [49]:
data_rolling

,key_press_count_count,delete_press_count_count,enter_press_count_count,shift_press_count_count,space_press_count_count,press_pause_count_count,pause_length_count,keystroke_time_count,press_to_press_time_count,click_count_count,...,press_to_press_time_skew,click_count_skew,double_click_count_skew,rotation_distance_skew,rotation_speed_skew,event_count_skew,gesture_distance_skew,gesture_speed_skew,gesture_deviation_skew,gesture_duration_seconds_skew
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-02-10 12:13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-02-10 12:14:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,0.113522,-1.384251,0.568562,0.291617,0.511355
2020-02-10 12:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,...,NaN,-1.732051,1.732051,NaN,NaN,-0.369285,-0.615757,0.829430,0.622644,0.092964
2020-02-10 12:16:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,-0.889037,2.000000,NaN,NaN,-0.272651,-0.480838,0.968055,0.751619,0.128085
2020-02-10 12:17:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,-0.598415,2.236068,NaN,NaN,-0.198359,-0.559129,0.886900,0.705495,0.114300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-26 11:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-26 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-26 12:01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
power_data = get_power(device, d1)

In [51]:
power_data_rolling = power_data.rolling('15Min', min_periods=1, win_type=None).mean()
power_data_rolling

,power
timestamp,
2020-02-07 10:06:27,0.0
2020-02-07 10:06:28,0.0
2020-02-07 10:06:29,0.0
2020-02-07 10:06:30,0.0
2020-02-07 10:06:31,0.0
...,...
2020-05-18 23:59:55,0.0
2020-05-18 23:59:56,0.0
2020-05-18 23:59:57,0.0


In [61]:
data_merged = data_rolling.merge(power_data_rolling, how='left', left_index=True, right_index=True)
data_merged = data_merged[data_merged.power >= 0.95]
data_merged

,key_press_count_count,delete_press_count_count,enter_press_count_count,shift_press_count_count,space_press_count_count,press_pause_count_count,pause_length_count,keystroke_time_count,press_to_press_time_count,click_count_count,...,click_count_skew,double_click_count_skew,rotation_distance_skew,rotation_speed_skew,event_count_skew,gesture_distance_skew,gesture_speed_skew,gesture_deviation_skew,gesture_duration_seconds_skew,power
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-02-10 12:13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.992222
2020-02-10 12:14:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,0.113522,-1.384251,0.568562,0.291617,0.511355,0.992222
2020-02-10 12:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,...,-1.732051,1.732051,NaN,NaN,-0.369285,-0.615757,0.829430,0.622644,0.092964,0.992222
2020-02-10 12:16:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,-0.889037,2.000000,NaN,NaN,-0.272651,-0.480838,0.968055,0.751619,0.128085,0.992222
2020-02-10 12:17:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,-0.598415,2.236068,NaN,NaN,-0.198359,-0.559129,0.886900,0.705495,0.114300,0.992222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-20 00:57:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
2020-03-20 00:58:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
2020-03-20 00:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000


In [62]:
data_merged = data_merged.drop(columns='power')

In [63]:
data_merged['column_sum'] = data_merged.fillna(0).sum(axis=1)

In [64]:
data_merged[data_merged.column_sum != 0.0]

,key_press_count_count,delete_press_count_count,enter_press_count_count,shift_press_count_count,space_press_count_count,press_pause_count_count,pause_length_count,keystroke_time_count,press_to_press_time_count,click_count_count,...,click_count_skew,double_click_count_skew,rotation_distance_skew,rotation_speed_skew,event_count_skew,gesture_distance_skew,gesture_speed_skew,gesture_deviation_skew,gesture_duration_seconds_skew,column_sum
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-02-10 12:13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.761197
2020-02-10 12:14:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,0.113522,-1.384251,0.568562,0.291617,0.511355,63.131293
2020-02-10 12:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,...,-1.732051,1.732051,NaN,NaN,-0.369285,-0.615757,0.829430,0.622644,0.092964,105.106782
2020-02-10 12:16:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,-0.889037,2.000000,NaN,NaN,-0.272651,-0.480838,0.968055,0.751619,0.128085,128.871226
2020-02-10 12:17:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,-0.598415,2.236068,NaN,NaN,-0.198359,-0.559129,0.886900,0.705495,0.114300,164.926313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-19 16:58:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.814089
2020-03-19 16:59:00,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,0.613440,0.776517,1.404018,-0.120032,0.280507,44.784217
2020-03-19 17:00:00,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,-0.749675,-0.168150,-0.371769,-0.714789,-1.106441,73.112836


In [65]:
data_merged.drop(columns=['column_sum']).dropna(how='all')

,key_press_count_count,delete_press_count_count,enter_press_count_count,shift_press_count_count,space_press_count_count,press_pause_count_count,pause_length_count,keystroke_time_count,press_to_press_time_count,click_count_count,...,press_to_press_time_skew,click_count_skew,double_click_count_skew,rotation_distance_skew,rotation_speed_skew,event_count_skew,gesture_distance_skew,gesture_speed_skew,gesture_deviation_skew,gesture_duration_seconds_skew
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-02-10 12:13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-02-10 12:14:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,0.113522,-1.384251,0.568562,0.291617,0.511355
2020-02-10 12:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,...,NaN,-1.732051,1.732051,NaN,NaN,-0.369285,-0.615757,0.829430,0.622644,0.092964
2020-02-10 12:16:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,-0.889037,2.000000,NaN,NaN,-0.272651,-0.480838,0.968055,0.751619,0.128085
2020-02-10 12:17:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,-0.598415,2.236068,NaN,NaN,-0.198359,-0.559129,0.886900,0.705495,0.114300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-19 16:58:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-19 16:59:00,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,0.613440,0.776517,1.404018,-0.120032,0.280507
2020-03-19 17:00:00,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,-0.749675,-0.168150,-0.371769,-0.714789,-1.106441
